In [2]:
using Pkg
pkg"activate ."
pkg"instantiate"

  Activating project at `C:\Users\ACER`


### Modules

Namespaces where a method or a type reside. Using a method or type outside the module may require explicit qualification. 

In [3]:
module m1
    function f()
        println("Defined as m1.f")
    end
    function g()
        println("Defined as m1.g")
    end
    function h()
        println("Defined as m1.h")
    end
    export f
end

module m2
    function f()
        println("Defined as m2.f")
    end
    function g()
        println("Defined as m2.g")
    end
    export g
end

Main.m2

In [4]:
names(m1)

2-element Vector{Symbol}:
 :f
 :m1

In [5]:
names(m2)

2-element Vector{Symbol}:
 :g
 :m2

In [6]:
f()

LoadError: UndefVarError: f not defined

In [7]:
m1.f()

Defined as m1.f


In [8]:
m2.f()

Defined as m2.f


In [9]:
using .m1

In [10]:
f()

Defined as m1.f


In [11]:
g()

LoadError: UndefVarError: g not defined

In [12]:
using Main.m2

In [13]:
g()

Defined as m2.g


In [14]:
using .m1: g

In [15]:
g()

Defined as m2.g


In [16]:
using .m1: h

In [17]:
h()

Defined as m1.h


## 9.2 Standard Modules

`Core`, `Base` and `Main` the bare minimum modules of a Julia shell. `m1` and `m2` here are submodules of `Main`.

In [18]:
varinfo()

| name |      size | summary |
|:---- | ---------:|:------- |
| Base |           | Module  |
| Core |           | Module  |
| Main |           | Module  |
| m1   | 6.380 KiB | Module  |
| m2   | 5.526 KiB | Module  |


In [19]:
x = 1

1

In [20]:
varinfo()

| name |      size | summary |
|:---- | ---------:|:------- |
| Base |           | Module  |
| Core |           | Module  |
| Main |           | Module  |
| m1   | 6.380 KiB | Module  |
| m2   | 5.526 KiB | Module  |
| x    |   8 bytes | Int64   |


## 9.3 System

The submodule that interacts with the operating system. System command execution being one of them. 

In [21]:
run(`cmd /c dir`)

 Volume in drive C is Acer
 Volume Serial Number is 9EB3-C855

 Directory of C:\Users\ACER

30-09-2022  15:00    <DIR>          .
28-10-2021  20:32    <DIR>          ..
30-09-2022  14:33    <DIR>          .conda
26-07-2022  15:38                25 .condarc
26-07-2022  15:38    <DIR>          .continuum
10-05-2021  17:36    <DIR>          .docker
30-09-2022  14:36    <DIR>          .ipynb_checkpoints
26-07-2022  15:40    <DIR>          .ipython
17-09-2022  23:39    <DIR>          .julia
14-09-2022  00:19    <DIR>          .jupyter
12-09-2022  16:19    <DIR>          .matplotlib
06-07-2021  20:16    <DIR>          3D Objects
25-07-2020  19:59    <DIR>          ansel
14-09-2022  19:50         4,863,260 brownsville texas python assignmet.ipynb
15-09-2022  12:08            43,544 Chapter-2 Data Types.ipynb
15-09-2022  12:17            13,642 Chapter-3 Conditions Control Flows and Iterations.ipynb
17-09-2022  23:54             3,896 Chapter-4 Functions and Methods jk.ipynb
17-09-2022  23:52 

Process(`cmd /c dir`, ProcessExited(0))

In [22]:
Sys.isunix(), Sys.iswindows()

(false, true)

## 9.4 Filesystem

File organization and manipulation (not read and write). 

In [23]:
old_dir = pwd()

"C:\\Users\\ACER"

In [24]:
cd("..")

In [25]:
pwd()

"C:\\Users"

In [26]:
cd(old_dir)

In [27]:
pwd()

"C:\\Users\\ACER"

In [28]:
cd(".."); pwd()

"C:\\Users"

In [29]:
readdir()  #Same as readdir(pwd())

6-element Vector{String}:
 "ACER"
 "All Users"
 "Default"
 "Default User"
 "Public"
 "desktop.ini"

In [30]:
for (root, dirs, files) in walkdir("Chapter-09")
    println("Directories in $root")
    for dir in dirs
        println(joinpath(root, dir)) # path to directories
    end
    println("Files in $root")
    for file in files
        println(joinpath(root, file)) # path to files
    end
end

In [31]:
cd(old_dir); pwd()

"C:\\Users\\ACER"

In [32]:
fs = stat("Chapter-9 Standard Libraries.ipynb")

StatStruct for "Chapter-9 Standard Libraries.ipynb"
   size: 69353 bytes
 device: 2662582357
  inode: 15903
   mode: 0o100666 (-rw-rw-rw-)
  nlink: 1
    uid: 0
    gid: 0
   rdev: 0
  blksz: 4096
 blocks: 136
  mtime:  (5 hours ago)
  ctime:  (5 hours ago)

#### Tasks

In [33]:
t = Task() do 
    sleep(10)
    println("done")
end

Task (runnable) @0x000000007248c5e0

In [34]:
schedule(t)

Task (runnable) @0x000000007248c5e0

In [35]:
t = Task() do 
    sleep(10)
    println("done")
end

Task (runnable) @0x000000000c12c7d0

In [36]:
schedule(t); wait(t)

done
done


In [37]:
t = @task begin
    sleep(5)
    println("done")
end

Task (runnable) @0x000000000c12c9c0

In [38]:
@sync  begin
    sleep(5)
    println("done")
end

done


#### Channels

In [39]:
c = Channel(8)

Channel{Any}(8) (empty)

In [40]:
c = Channel{Int}(4)

Channel{Int64}(4) (empty)

In [41]:
@async begin
    for i = 1:100
        println("Adding $i to channel")
        put!(c, i)
    end
end

@async begin
    for i = 1:100
        v = take!(c)
        println("Removing value: $v")
    end
end

Task (runnable) @0x000000007248cbb0

Adding 1 to channel


In [42]:
close(c)

Adding 2 to channel
Adding 3 to channel
Removing value: 1
Removing value: 2

### Multithreading

Julia can be made to run on a multiple threads mode. In such a conditions the tasks can be parallelized on these threads. Synchronization objects can be used across the executing threads. 

In [43]:
Threads.nthreads()

1

In [44]:
Threads.@spawn for i=1:100
    sleep(1)
    println("Step: $i")
end

Task (runnable) @0x000000000a2fd370

### Default Stream

`stdin`, `stdout` and `stderr` are the standard streams for input, output and error in most processes. `stdin` integration with Jupyter is limited. All those samples may best be reviewed in a REPL console. 

In [45]:
write(stdout, "Hello World")

Hello World

11

In [46]:
write(stdout, "\u2200 x \u2203 y")

∀ x ∃ y

11

### File I/O

Files are also accessed as `IO` streams for both read and write operations. However, the `IO` streams are obtained by `open` call. Any open stream must be closed with a `close` function call. 

In [47]:
write("hello.txt", "Hello World")

11

In [48]:
fd = open("hello.txt", "r");
data = read(fd);

In [49]:
write(fd, "This is second line")

LoadError: ArgumentError: write failed, IOStream is not writeable

In [50]:
close(fd)
write(stdout, data)

Hello World

11

In [51]:
open("hello.txt", "r") do fd
    data = read(fd)
    write(stdout, data)
end

Hello World

11

In [52]:
write("hello.txt", "Hello World");
data = read("hello.txt");
write(stdout, data);

Hello World

Files are opened for `read` or `write`. A file opened for reading cannot be written into.  

In [53]:
open("hello.txt", "r") do fd
    while !eof(fd)
        b = read(fd, 1)
        write(stdout, b)
    end
end
rm("hello.txt")

Hello World

### Delimited Files

Generalized command separated value (CSV) kind of files. Used extensively in spreadsheets. Also interfaced in data science applications as an easier way to share data. 

In [54]:
using DelimitedFiles
x = collect(1:100);
y = Float64.(collect(101:200));
open("values.csv", "w") do f
    println(f, "X,Y")
    writedlm(f, [x y], ',')
end

In [55]:
data, header = readdlm("values.csv", ',', header=true);

Step: 1


In [56]:
size(data, 1)

100

In [57]:
header

1×2 Matrix{AbstractString}:
 "X"  "Y"

In [58]:
rm("values.csv")

### Network 

Just like files are opened with open to obtain the `IO` stream to interface, sockets are equivalent network communication end points to obtain connection streams. However, if you could use download method to provide a URL and download files. 

In [59]:
download("https://github.com/JuliaLang/julia/releases/download/v1.5.3/julia-1.5.3.tar.gz", "julia-1.5.3.tar.gz")

Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8


"julia-1.5.3.tar.gz"

In [60]:
using Sockets
@async begin
    server = listen(3000)
    while true
        s = accept(server)
        @async while isopen(s)
            msg = readline(s, keep=true)
            write(s, "Server Response: $msg")
        end  
    end
end

Task (runnable) @0x000000000c243a30

In [61]:
c = connect(3000)
@async while isopen(c)
    println(stdout, readline(c, keep=true))
end

Task (runnable) @0x0000000073df8010

In [62]:
for i = 1:5
    println(c, "Client Msg id: $i")
end

Server Response: Client Msg id: 1

Server Response: Client Msg id: 2



In [63]:
close(c)

Server Response: Client Msg id: 3

Step: 9


### Memory I/O

Strings cannot be manipulated. It may help to create a byte array for `IO` in the memory and use that for text manipulations. In the end the byte array can be converted to a String. 

In [64]:
io = IOBuffer()
println(io, "This is Line 1")
println(io, "This is Line 2")
str = String(take!(io))

"This is Line 1\nThis is Line 2\n"

In [65]:
A = fill(10, (5, 20))
A[2, 3] = 20
open("mmap.bin", "w+") do fd
    write(fd, size(A, 1))
    write(fd, size(A, 2))
    write(fd, A)
end

800

Memory mapping is reverse problem. A file can be mapped to memory and accessed selectively page by page. This way the whole file need not be loaded in the memory. 

In [66]:
using Mmap
fd = open("mmap.bin", "r")
nr = read(fd, Int)
nc = read(fd, Int)
A2 = Mmap.mmap(fd, Matrix{Int}, (nr, nc))
println("A2[1, 1]: ", A2[1, 1], " A2[2, 3]: ", A2[2, 3])
close(fd)

A2[1, 1]: 10 A2[2, 3]: 20


In [67]:
A2 = nothing
GC.gc()
rm("mmap.bin")

## 9.7 Constants

There are many system and Julia related constants. Mathematical constants are also defined in the standard library. 

In [68]:
VERSION

v"1.8.1"

In [69]:
C_NULL

Ptr{Nothing} @0x0000000000000000

In [70]:
Sys.BINDIR

Step: 10


"C:\\Users\\ACER\\AppData\\Local\\Programs\\Julia-1.8.1\\bin"

In [71]:
Sys.CPU_THREADS

8

In [72]:
Sys.WORD_SIZE

64

In [73]:
Sys.KERNEL

:NT

In [74]:
Sys.MACHINE

"x86_64-w64-mingw32"

In [75]:
Sys.ARCH

:x86_64

In [76]:
Sys.ENV

Base.EnvDict with 47 entries:
  "=C:"                     => "C:\\Users\\ACER"
  "ALLUSERSPROFILE"         => "C:\\ProgramData"
  "APPDATA"                 => "C:\\Users\\ACER\\AppData\\Roaming"
  "COLUMNS"                 => "80"
  "COMMONPROGRAMFILES"      => "C:\\Program Files\\Common Files"
  "COMMONPROGRAMFILES(X86)" => "C:\\Program Files (x86)\\Common Files"
  "COMMONPROGRAMW6432"      => "C:\\Program Files\\Common Files"
  "COMPUTERNAME"            => "LAPTOP-DJ4KEKK9"
  "COMSPEC"                 => "C:\\WINDOWS\\system32\\cmd.exe"
  "DRIVERDATA"              => "C:\\Windows\\System32\\Drivers\\DriverData"
  "HOMEDRIVE"               => "C:"
  "HOMEPATH"                => "\\Users\\ACER"
  "JULIA"                   => "C:\\Users\\ACER\\AppData\\Local\\Programs\\Juli…
  "LINES"                   => "30"
  "LOCALAPPDATA"            => "C:\\Users\\ACER\\AppData\\Local"
  "LOGONSERVER"             => "\\\\LAPTOP-DJ4KEKK9"
  "NUMBER_OF_PROCESSORS"    => "8"
  "ONEDRIVE"              

In [77]:
MathConstants.pi

Step: 11


π = 3.1415926535897...

In [78]:
MathConstants.e

ℯ = 2.7182818284590...

## 9.8 Notable Modules

Miscellaneous modules of Julia library. 

### Dates

User renderable date and time representations. In the system the date is just a continuously increasing counter from a datum. These functions make meaningful period associations and provide the correct date and time. 

In [79]:
using Dates

In [80]:
now()

2022-09-30T20:43:58.492

In [81]:
DateTime(2020)

Step: 12


2020-01-01T00:00:00

In [82]:
t = DateTime(2020, 10, 31, 01, 02, 03)

2020-10-31T01:02:03

In [83]:
Date(2020)

2020-01-01

In [84]:
d = Date(t)

2020-10-31

In [85]:
DateTime(d)

2020-10-31T00:00:00

In [86]:
t > d

true

In [87]:
DateTime(2020) < now()

true

In [88]:
DateTime(2020) == Date(2020)

true

In [89]:
p = Month(3) + Day(10) + Hour(15)

3 months, 10 days, 15 hours

Step: 13


In [90]:
typeof(p)

Dates.CompoundPeriod

In [91]:
now() + p

2023-01-10T11:44:00.872

In [92]:
Date(now())+ Day(300)

2023-07-27

In [93]:
now() - DateTime(2020)

86733840955 milliseconds

In [94]:
now() - Date(2020)

LoadError: MethodError: no method matching -(::DateTime, ::Date)
[0mClosest candidates are:
[0m  -([91m::StridedArray{<:Union{Dates.CompoundPeriod, Period}}[39m, ::TimeType) at C:\Users\ACER\AppData\Local\Programs\Julia-1.8.1\share\julia\stdlib\v1.8\Dates\src\deprecated.jl:26
[0m  -(::TimeType, [91m::Dates.CompoundPeriod[39m) at C:\Users\ACER\AppData\Local\Programs\Julia-1.8.1\share\julia\stdlib\v1.8\Dates\src\periods.jl:400
[0m  -(::T, [91m::T[39m) where T<:TimeType at C:\Users\ACER\AppData\Local\Programs\Julia-1.8.1\share\julia\stdlib\v1.8\Dates\src\arithmetic.jl:9
[0m  ...

In [95]:
Date(now()) - Date(2020)

Step: 14


1003 days

In [96]:
Year(now())

2022 years

In [97]:
Month(now())

9 months

In [98]:
Day(now())

30 days

In [99]:
Week(now())

39 weeks

In [100]:
year(now())

Step: 15


2022

In [101]:
tonext(today()) do d
    dayofweek(d) == Thursday &&
    dayofweekofmonth(d) == 4 &&
    month(d) == November
end

2022-11-24

### Logging

Long running processes or servers running in Julia need to provide consistent and standard log reporting. This module helps create such reports. 

In [102]:
@debug "This is a debug message $(sum(rand(100)))"

In [103]:
@info "This message is just informational"

┌ Info: This message is just informational
└ @ Main In[103]:1


In [104]:
@warn "This is a warning message"

┌ Warning: This is a warning message
└ @ Main In[104]:1


In [105]:
@error "This is an error message"

┌ Error: This is an error message
└ @ Main In[105]:1


In [106]:
A = ones(3, 4)
@info "A is all ones" A

┌ Info: A is all ones
│   A = [1.0 1.0 1.0 1.0; 1.0 1.0 1.0 1.0; 1.0 1.0 1.0 1.0]
└ @ Main In[106]:2


The `Logging` module provides interfaces to implement complete loggers. It also provides the ability to override the global logger with a different local one. 

In [107]:
using Logging

In [108]:
open("file.log", "w") do f
    with_logger(SimpleLogger(f)) do
        @info "This is my simple info log"
        @error "This is an error written to file"
        @warn "This is a warning message"
        @info "Current logger is same as global logger" current_logger() == global_logger()
    end
end
write(stdout, read("file.log"));

┌ Info: This is my simple info log
└ @ Main In[108]:3
┌ Error: This is an error written to file
└ @ Main In[108]:4
┌ Warning: This is a warning message
└ @ Main In[108]:5
┌ Info: Current logger is same as global logger
│   current_logger() == global_logger() = false
└ @ Main In[108]:6


In [109]:
rm("file.log")

In [110]:
@info "Current logger is same as global logger" current_logger() == global_logger()

┌ Info: Current logger is same as global logger
│   current_logger() == global_logger() = true
└ @ Main In[110]:1


### Statistics

This module provides simple functions for central tendencies like mean, median and standard deviations etc. Functions for correlations and covariances are also provided. However, the treatment of statistics in thi module are highly rudimentary. 

In [111]:
using Statistics

In [112]:
mean(1:10)

5.5

In [113]:
median(1:10)

Step: 16


5.5

In [114]:
std(1:10)

3.0276503540974917

In [115]:
var(1:10)

9.166666666666666

In [116]:
cor(1:10, rand(10))

-0.12338368390746393

In [117]:
cor(1:10, 11:20)

1.0

In [118]:
cov(1:10, rand(10))

-0.08904586957103043

### Random Numbers

Provides methods for random number generation and other permutation and combination functions. 

In [119]:
using Random

In [120]:
rand(Int, 2)

2-element Vector{Int64}:
 -2953195918062992480
  7533823295185910339

In [121]:
rand(2, 3)

Step: 17


2×3 Matrix{Float64}:
 0.159478   0.595196  0.440281
 0.0281745  0.403192  0.577707

In [122]:
rand!(zeros(2, 3))

2×3 Matrix{Float64}:
 0.943004  0.284251  0.54992
 0.377371  0.820076  0.603596

In [123]:
rand(1:4, (2, 3))

2×3 Matrix{Int64}:
 3  3  3
 1  3  2

Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Step: 50
Step: 51
Step: 52
Step: 53
Step: 54
Step: 55
Step: 56
Step: 57
Step: 58
Step: 59
Step: 60
Step: 61
Step: 62
Step: 63
Step: 64
Step: 65
Step: 66
Step: 67
Step: 68
Step: 69
Step: 70
Step: 71
Step: 72
Step: 73
Step: 74
Step: 75
Step: 76
Step: 77
Step: 78
Step: 79
Step: 80
Step: 81
Step: 82
Step: 83
Step: 84
